In [ ]:
using ITensors
using JLD2
using Plots
using LinearAlgebra
using Printf

gr() # Plotting backend

function analyze_hpc_results()
    filename = "dmrg_hpc_results.jld2"
    if !isfile(filename)
        println("Error: $filename not found.")
        return
    end
    
    println("Loading HPC Results...")
    results_db = load(filename, "results_db")
    
    # Collect available N values
    Ns = sort(unique([k[1] for k in keys(results_db)]))
    
    println("\n=== ARTIFACT VERIFICATION (Fidelity Check) ===")
    println("Hypothesis: The high bond dimension in noisy systems is an artifact.")
    println("Method: Truncate Noisy MPS to Clean MPS size. If Overlap ~ 1.0, Hypothesis is TRUE.")
    
    for N in Ns
        # 1. Get Clean Baseline
        if !haskey(results_db, (N, 0.0))
            println("Skipping N=$N (Missing baseline data)")
            continue
        end
        
        clean_data = results_db[(N, 0.0)]
        psi_clean = clean_data["mps"]
        chi_clean = maxlinkdim(psi_clean)
        
        println("\nSystem Size N = $N (Clean BD = $chi_clean)")
        println("-"^60)
        @printf("%-10s | %-12s | %-12s | %-10s\n", "Sigma", "Noisy BD", "Truncated F", "Result")
        println("-"^60)
        
        # 2. Compare against Noisy variants
        noisy_sigmas = sort([k[2] for k in keys(results_db) if k[1] == N && k[2] > 0.0])
        
        for σ in noisy_sigmas
            noisy_data = results_db[(N, σ)]
            psi_noisy = noisy_data["mps"]
            chi_noisy = maxlinkdim(psi_noisy)
            
            # --- FIDELITY TEST ---
            # Create a copy to truncate
            psi_test = copy(psi_noisy)
            
            # Truncate strictly to the clean bond dimension
            # We use the clean system's max bond dimension as the limit
            truncate!(psi_test; maxdim=chi_clean, cutoff=1E-14)
            
            # Calculate Overlap
            # |<Clean | Noisy_Truncated>|^2
            ov = abs(inner(psi_clean, psi_test))
            fidelity = ov^2
            
            status = fidelity > 0.99 ? "PASS" : "FAIL"
            
            @printf("%-10.3f | %-12d | %-12.6f | %-10s\n", σ, chi_noisy, fidelity, status)
        end
        println("-"^60)
    end
    
    # --- PLOTTING ---
    # Plotting the spectrum for the largest available N to visualize the "Sea of Noise"
    N_plot = maximum(Ns)
    sigmas = sort([k[2] for k in keys(results_db) if k[1] == N_plot])
    
    p = plot(title="Schmidt Spectrum (N=$N_plot)", 
             xlabel="Singular Value Index", ylabel="Probability (λ²)",
             yaxis=:log10, legend=:bottomleft, dpi=300)
             
    for σ in sigmas
        spec = results_db[(N_plot, σ)]["spectrum"]
        # Sort descending
        spec = sort(spec, rev=true)
        
        lbl = σ == 0.0 ? "Clean (σ=0)" : "Noisy (σ=$σ)"
        ls = σ == 0.0 ? :solid : :dash
        
        plot!(p, spec, label=lbl, linestyle=ls, lw=2)
    end
    
    savefig(p, "spectrum_N$(N_plot)_hpc.png")
    println("\nSpectrum plot saved to spectrum_N$(N_plot)_hpc.png")
end

analyze_hpc_results()